In [1]:
import pickle
import pandas as pd
import numpy as np
import json
import re
import time
import ast

### 1. Load Dataset

In [3]:
dataset_df = pd.read_csv('match_data_small.csv', index_col=0)
dataset_df = dataset_df.reset_index(drop=True)
dataset_df

,Unnamed: 0.1,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities,status
0,0,4.937024e+09,KR,1.611190e+12,1764.0,420.0,11.0,13.0,11.2.353.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
1,0,4.936849e+09,KR,1.611170e+12,1981.0,420.0,11.0,13.0,11.1.352.5559,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
2,0,4.936828e+09,KR,1.611169e+12,911.0,420.0,11.0,13.0,11.1.352.5559,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
3,0,4.936875e+09,KR,1.611167e+12,1562.0,420.0,11.0,13.0,11.1.352.5559,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
4,0,4.936813e+09,KR,1.611164e+12,1699.0,420.0,11.0,13.0,11.1.352.5559,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10949,NaN,4.992874e+09,KR,1.612980e+12,1140.0,420.0,11.0,13.0,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
10950,NaN,4.992778e+09,KR,1.612977e+12,1586.0,420.0,11.0,13.0,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
10951,NaN,4.992782e+09,KR,1.612975e+12,1543.0,420.0,11.0,13.0,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
10952,NaN,4.992737e+09,KR,1.612974e+12,938.0,420.0,11.0,13.0,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN


In [4]:
print('Game version:', dataset_df['gameVersion'].unique())
print('QueueId:', dataset_df['queueId'].unique(), end='\n\n')  # 420: solo-rank, 430: blind, 440: free-rank
dataset_df.reset_index()

Game version: ['11.2.353.8505' '11.1.352.5559' '11.3.357.5376' nan '11.3.356.7268'
 '11.1.351.8352' '10.25.350.1724' '10.23.343.2581' '10.22.341.643'
 '10.20.338.336' '10.19.336.4199' '10.16.330.9186' '10.15.330.344'
 '10.15.328.8759' '10.14.327.8505' '10.14.327.1684' '10.12.324.5925'
 '10.10.320.3039' '10.25.348.1797' '10.24.345.8390' '10.21.339.2173'
 '10.19.335.4706' '10.18.333.8889' '10.20.337.6669']
QueueId: [ 420.  430.  900.  450.   nan  440.  850. 1300.  700.  840.  830.]



,index,Unnamed: 0.1,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities,status
0,0,0,4.937024e+09,KR,1.611190e+12,1764.0,420.0,11.0,13.0,11.2.353.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
1,1,0,4.936849e+09,KR,1.611170e+12,1981.0,420.0,11.0,13.0,11.1.352.5559,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
2,2,0,4.936828e+09,KR,1.611169e+12,911.0,420.0,11.0,13.0,11.1.352.5559,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
3,3,0,4.936875e+09,KR,1.611167e+12,1562.0,420.0,11.0,13.0,11.1.352.5559,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
4,4,0,4.936813e+09,KR,1.611164e+12,1699.0,420.0,11.0,13.0,11.1.352.5559,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10949,10949,NaN,4.992874e+09,KR,1.612980e+12,1140.0,420.0,11.0,13.0,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
10950,10950,NaN,4.992778e+09,KR,1.612977e+12,1586.0,420.0,11.0,13.0,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
10951,10951,NaN,4.992782e+09,KR,1.612975e+12,1543.0,420.0,11.0,13.0,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN
10952,10952,NaN,4.992737e+09,KR,1.612974e+12,938.0,420.0,11.0,13.0,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",NaN


In [12]:
max(dataset_df['gameCreation'])

1613372737629.0

In [4]:
ast.literal_eval(dataset_df['teams'][0])

[{'teamId': 100,
  'win': 'Fail',
  'firstBlood': False,
  'firstTower': True,
  'firstInhibitor': False,
  'firstBaron': False,
  'firstDragon': False,
  'firstRiftHerald': True,
  'towerKills': 4,
  'inhibitorKills': 0,
  'baronKills': 0,
  'dragonKills': 2,
  'vilemawKills': 0,
  'riftHeraldKills': 2,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 777, 'pickTurn': 1},
   {'championId': 360, 'pickTurn': 2},
   {'championId': 60, 'pickTurn': 3},
   {'championId': 412, 'pickTurn': 4},
   {'championId': 76, 'pickTurn': 5}]},
 {'teamId': 200,
  'win': 'Win',
  'firstBlood': True,
  'firstTower': False,
  'firstInhibitor': True,
  'firstBaron': True,
  'firstDragon': True,
  'firstRiftHerald': False,
  'towerKills': 9,
  'inhibitorKills': 2,
  'baronKills': 2,
  'dragonKills': 2,
  'vilemawKills': 0,
  'riftHeraldKills': 0,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 3, 'pickTurn': 6},
   {'championId': 412, 'pickTurn': 7},
   {'championId': 58, 'pickTurn': 8},
   {'cha

In [5]:
ast.literal_eval(dataset_df['participants'][0])[0]

{'participantId': 1,
 'teamId': 100,
 'championId': 72,
 'spell1Id': 4,
 'spell2Id': 11,
 'stats': {'participantId': 1,
  'win': False,
  'item0': 6664,
  'item1': 3742,
  'item2': 3193,
  'item3': 3041,
  'item4': 3067,
  'item5': 3158,
  'item6': 3364,
  'kills': 8,
  'deaths': 5,
  'assists': 9,
  'largestKillingSpree': 7,
  'largestMultiKill': 2,
  'killingSprees': 1,
  'longestTimeSpentLiving': 873,
  'doubleKills': 1,
  'tripleKills': 0,
  'quadraKills': 0,
  'pentaKills': 0,
  'unrealKills': 0,
  'totalDamageDealt': 220941,
  'magicDamageDealt': 109982,
  'physicalDamageDealt': 91853,
  'trueDamageDealt': 19105,
  'largestCriticalStrike': 0,
  'totalDamageDealtToChampions': 18290,
  'magicDamageDealtToChampions': 9700,
  'physicalDamageDealtToChampions': 7211,
  'trueDamageDealtToChampions': 1378,
  'totalHeal': 8659,
  'totalUnitsHealed': 1,
  'damageSelfMitigated': 64592,
  'damageDealtToObjectives': 30995,
  'damageDealtToTurrets': 74,
  'visionScore': 28,
  'timeCCingOthers'

In [6]:
ast.literal_eval(dataset_df['participantIdentities'][0])

[{'participantId': 1,
  'player': {'platformId': 'KR',
   'accountId': 'bvk5F2Wemv0iGPn_CCuEdHYkx3xhw_XsnlQSnsOaV-ECe74',
   'summonerName': 'Iknu',
   'summonerId': 'm1B4wC1T5ya97AFB-0KARhMDrAeaPk2eLoWjVsxhuImNstM',
   'currentPlatformId': 'KR',
   'currentAccountId': 'bvk5F2Wemv0iGPn_CCuEdHYkx3xhw_XsnlQSnsOaV-ECe74',
   'matchHistoryUri': '/v1/stats/player_history/KR/202090712',
   'profileIcon': 4568}},
 {'participantId': 2,
  'player': {'platformId': 'KR',
   'accountId': 'Qc_Q1u8vY7HyhYBKhrmxVnisYNBbKkWfcDn2rioN-cEh',
   'summonerName': 'twenty2',
   'summonerId': 'vhETIHu2Y1OmFu9RaAzUdb0X8U2KwBf1PDypUq0g702K4rs',
   'currentPlatformId': 'KR',
   'currentAccountId': 'Qc_Q1u8vY7HyhYBKhrmxVnisYNBbKkWfcDn2rioN-cEh',
   'matchHistoryUri': '/v1/stats/player_history/KR/7373632',
   'profileIcon': 4782}},
 {'participantId': 3,
  'player': {'platformId': 'KR',
   'accountId': 'b-qCxqZ3rlPp64Wff0VbD1uYt0yyQin74zagfoSnU5I5E7c',
   'summonerName': '나한테만',
   'summonerId': 'OjY4GKQ3hj2pWQ1lOg

### 2. Preprocess Dataset

#### Extract the unique matches

In [7]:
dataset_df = dataset_df.drop_duplicates()
len(dataset_df)

14545

#### Consider only Ranked games

In [8]:
dataset_df = dataset_df[(dataset_df['queueId'] == 420) | (dataset_df['queueId'] == 440)]
len(dataset_df)

12965

#### Remain the necessary information

In [9]:
# queueId, gameVersion, teams, participants, participantIdentities
dataset_df = dataset_df[['gameVersion', 'teams', 'participants', 'participantIdentities']]
dataset_df = dataset_df.reset_index(drop=True)
dataset_df

,gameVersion,teams,participants,participantIdentities
0,11.3.356.7268,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,11.3.356.7268,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2,11.3.356.7268,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3,11.3.356.7268,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
4,11.3.356.7268,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
...,...,...,...,...
12960,11.1.352.5559,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
12961,11.1.352.5559,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
12962,11.1.352.5559,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
12963,11.1.352.5559,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


### 3. Split into train & test Dataset

In [10]:
div_version = '11.3.356.7268'

In [11]:
train_data = dataset_df[dataset_df['gameVersion'] < div_version].reset_index(drop=True)
test_data = dataset_df[dataset_df['gameVersion'] >= div_version].reset_index(drop=True)
print('Num train data:', len(train_data))
print('Num test data:', len(test_data))

Num train data: 11030
Num test data: 1935


#### Build dictionary with train Dataset

In [12]:
num_participants = 10
user_to_idx, idx_to_user = {}, {}
user_to_idx['UNK'] = 0
user_to_idx['MASK'] = 1
user_cnt = 2

# Build user-dictionary
for i in range(len(train_data)):
    participantIdentities = ast.literal_eval(train_data['participantIdentities'][i])
    for participant in participantIdentities:
        accountId = participant['player']['accountId']
        if accountId not in user_to_idx:
            user_to_idx[accountId] = user_cnt
            user_cnt += 1
            
for key, value in user_to_idx.items():
    idx_to_user[value] = key
    
len(user_to_idx)

19514

In [13]:
champion_to_idx, idx_to_champion = {}, {}
champion_to_idx['UNK'] = 0
champion_to_idx['MASK'] = 1
champion_cnt = 2

# Build champion-dictionary
for i in range(len(train_data)):
    participants = ast.literal_eval(train_data['participants'][i])
    for participant in participants:
        championId = participant['championId']
        if championId not in champion_to_idx:
            champion_to_idx[championId] = champion_cnt
            champion_cnt += 1
            
for key, value in champion_to_idx.items():
    idx_to_champion[value] = key
    
len(champion_to_idx)

156

In [14]:
version_to_idx, idx_to_version = {}, {}
version_to_idx['UNK'] = 0
version_cnt = 1

version_list = train_data['gameVersion'].unique()
for version in version_list:
    version_to_idx[version] = version_cnt
    version_cnt += 1

for key, value in version_to_idx.items():
    idx_to_version[value] = key
    
len(version_to_idx)

14

### 3. Create training Dataset

#### Dataset for Collaborative Filtering (=Interaction)

In [15]:
# user_champion_interaction_df (D): (N, C) matrix 
# N: number of users
# C: number of champions
# D_nc = proportion of the play

num_users = len(user_to_idx)
num_champions = len(champion_to_idx)

user_champion_matrix = np.zeros((num_users, num_champions))

In [16]:
for i in range(len(train_data)):
    participantIdentities = ast.literal_eval(train_data['participantIdentities'][i])
    participants = ast.literal_eval(train_data['participants'][i])
    for p_idx in range(num_participants):
        participant = participants[p_idx]
        participantIdentity = participantIdentities[p_idx]
        
        userIdx = user_to_idx[participantIdentity['player']['accountId']]
        championIdx = champion_to_idx[participant['championId']]
        
        user_champion_matrix[userIdx][championIdx] += 1

user_champion_matrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., 27., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

#### Dataset for Winning Prediction (=Match)

In [48]:
# Initialize dataframe
columns = ['win', 'gameVersion']
for participant_id in range(1,num_participants+1):
    columns.append('User'+str(participant_id)+'_id')
    columns.append('User'+str(participant_id)+'_lane')
    columns.append('User'+str(participant_id)+'_champion')
    columns.append('User'+str(participant_id)+'_ban')
match_data = pd.DataFrame(index=train_data.index, columns = columns)
match_data

,win,gameVersion,User1_id,User1_lane,User1_champion,User1_ban,User2_id,User2_lane,User2_champion,User2_ban,...,User8_champion,User8_ban,User9_id,User9_lane,User9_champion,User9_ban,User10_id,User10_lane,User10_champion,User10_ban
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
version_to_idx[gameVersion]

1

### Dataset for user-BERT4Rec

In [18]:
with open('categorical_ids.pickle', 'rb') as f:
    categorical_ids = pickle.load(f)
print(len(categorical_ids['user']))
print(len(categorical_ids['champion']))

7999
158


In [19]:
with open('./match_data.pickle', 'rb') as f:
    match_data = pickle.load(f)

match_data_df = pd.concat([match_data['train'], match_data['val'], match_data['test']]).reset_index(drop=True)
num_participants = 10
for participant_id in range(1,num_participants+1):
    match_data_df['User'+str(participant_id)+'_stat'] = 0
match_data_df

,win,version,time,User1_id,User1_lane,User1_champion,User1_ban,User1_team,User2_id,User2_lane,...,User1_stat,User2_stat,User3_stat,User4_stat,User5_stat,User6_stat,User7_stat,User8_stat,User9_stat,User10_stat
0,Fail,4,1612861073517,4,6,4,14,4,5,8,...,0,0,0,0,0,0,0,0,0,0
1,Win,4,1612858997685,11,5,14,119,4,6,7,...,0,0,0,0,0,0,0,0,0,0
2,Fail,4,1612856839482,17,8,20,49,4,18,4,...,0,0,0,0,0,0,0,0,0,0
3,Fail,4,1612853859870,3,7,32,92,4,23,8,...,0,0,0,0,0,0,0,0,0,0
4,Win,4,1612851830818,3,5,40,119,4,28,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93849,Win,4,1613303581951,7980,5,132,56,4,3,8,...,0,0,0,0,0,0,0,0,0,0
93850,Fail,4,1613316062066,3,6,26,19,4,3,5,...,0,0,0,0,0,0,0,0,0,0
93851,Fail,4,1613314804387,3,5,130,45,4,3,8,...,0,0,0,0,0,0,0,0,0,0
93852,Win,4,1613312605971,3,7,90,14,4,3,6,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from collections import defaultdict

num_participants = 10
UNK = 3

user_to_match_history = {}
for user_id in range(len(categorical_ids['user'])):
    user_to_match_history[user_id] = []

stat_id = 0
histories = defaultdict(list)
for match_idx in range(len(match_data_df)):
    match = match_data_df.iloc[match_idx]
    win = int(match['win'] == 'Win')
    time = match['time']
    bans = []
    for p_idx in range(num_participants):
        banIdx = match['User'+str(p_idx+1)+'_ban']
        bans.append(banIdx)
    
    for p_idx in range(num_participants):
        user =  match['User'+str(p_idx+1)+'_id']
        item = match['User'+str(p_idx+1)+'_champion']
        lane = match['User'+str(p_idx+1)+'_lane']
        # user belongs to blue-team so win should be identical
        user_stat_in_match = {}
        user_stat_in_match['id'] = stat_id
        user_stat_in_match['time'] = time
        user_stat_in_match['lane'] = lane
        user_stat_in_match['item'] = item
        user_stat_in_match['bans'] = bans
        
        if p_idx < 5:
            user_stat_in_match['win'] = win
        # user belongs to red-team so win should be reversed
        else:
            user_stat_in_match['win'] = 1-win
        if user != UNK:
            user_to_match_history[user].append(user_stat_in_match)
            histories['User'+str(p_idx+1)+'_stat'].append(stat_id)
            #match_data_df.loc[match_idx, 'User'+str(p_idx+1)+'_stat'] = stat_id
            stat_id += 1
        else:
            histories['User'+str(p_idx+1)+'_stat'].append(0)

In [14]:
for key, value in histories.items():
    match_data_df[key] = value
match_data_df

,win,version,time,User1_id,User1_lane,User1_champion,User1_ban,User1_team,User2_id,User2_lane,...,User1_stat,User2_stat,User3_stat,User4_stat,User5_stat,User6_stat,User7_stat,User8_stat,User9_stat,User10_stat
0,Fail,4,1609322998821,3,4,4,16,4,3,5,...,0,0,0,0,0,1,0,0,0,0
1,Win,4,1609238499229,3,4,14,33,4,3,7,...,0,0,0,2,0,3,0,0,0,0
2,Win,5,1612621059000,3,6,23,29,4,3,7,...,0,0,4,5,0,0,0,0,6,0
3,Fail,5,1612619789305,3,7,29,4,4,3,6,...,0,0,0,0,0,0,0,7,8,9
4,Fail,5,1612617603304,13,8,24,20,4,3,5,...,10,0,11,0,0,0,12,13,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43622,Fail,5,1613228394645,3,6,60,51,4,3,4,...,0,0,0,0,0,0,0,0,0,0
43623,Fail,5,1613226405632,3,7,55,109,4,3,4,...,0,0,0,0,0,0,0,0,0,0
43624,Fail,5,1613217326971,3,5,110,92,4,3,4,...,0,0,0,0,0,0,0,0,0,0
43625,Win,5,1613214949554,3,6,4,33,4,3,5,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# re-order the matches
for user, match_histories in user_to_match_history.items():
    user_to_match_history[user] = sorted(match_histories, key = lambda x : x['time'])

In [38]:
# check whether the item is propery ordered in sequence
for user, match_histories in user_to_match_history.items():
    cur_time = 0
    for match_history in match_histories:
        time = match_history['time']
        assert time > cur_time
        cur_time = time

In [40]:
with open('./user_history_data.pickle', 'wb') as f:
    pickle.dump(user_to_match_history, f)

In [ ]:
match_data['train'] = match_data_df[(match_data_df['time'] < args.train_end_time)].reset_index(drop=True)
match_data['val'] = match_data_df[(args.train_end_time <= match_data_df['time']) & 
                                  (match_data_df['time'] < args.val_end_time)].reset_index(drop=True)
match_data['test'] = match_data_df[(match_data_df['time'] > args.val_end_time)].reset_index(drop=True)